![title](https://docs.distributed-ci.io/logo.png)

# How to request DCI through Python

## Introduction

DCI or Distributed CI is a continuous integration project that aims to bring external teams inside an existing CI
framework. This by running CI on dedicated lab environments that are running in their data centers with their configurations and their applications.

DCI is base on a client/server model. The remote labs are the clients, the server centralize the jobs and collects the results.

The goal of this article is to give swift insight of how anyone can query the DCI control server.

## Set up your environment

We will use a Python virtual environment to install our DCI client library.

    $ virtualenv ~/pyvenv
    $ . ~/pyvenv/bin/activate
    $ pip install dciclient

You now need to load our dcirc file to be authenticated.

    $ . dcirc.sh

A simple way to validate everything is working properly is to list your remotecis:

    $ dcictl remoteci-list


Unless you get an error, everything is working fine.

## Our first  DCI query

First some vocabulary! In DCI, a branch is a major version on which we will attach the components that we want to test. For instance, I want to test all the snapshot of RHEL7.5, RHEL7.5 is my branch, RHEL7.5-20180115 is my component.

A job will be linked with a topic and one or more component.

In this first example, I've got a list of topic and I would like to know what are the last components that have been associated with a successful job.

So I declare my list of topics:

In [ ]:
topics = ['OSP8', 'OSP9', 'OSP10', 'OSP11', 'OSP12']

Now I need to create a DCI context. I load the api.context class and use the `build_dci_context()` to initialize a session based on my environment variables.

In [ ]:
import dciclient.v1.api.context as dci_context
ctx = dci_context.build_dci_context()

I know the name of the topics that I want to work with. But DCI will need their UUID. I create a light function that relies on `api.topic.list()` to find the correct topic and return.

In [ ]:
import dciclient.v1.api.topic as dci_topic
def topic_id_by_name(ctx, name):
    r = dci_topic.list(ctx, where='name:' + name, limit=1)
    if not len(r.json()['topics']):
        print('Topic %s not available.' % name)
        exit(1)
    return r.json()['topics'][0]['id']

Now I know the `topic_id`, I will be able to request the DCI server.

I want to filter the job list to unlike keep the ones linked to the
topic so I add a where filter on the topic_id.

I also would like to have some extra details about the associated
components, not just their UUID. So I use the `embed` key to ask DCI to
include the extra information for me.

Finally I use a limit to reduce the amount of information that will be
returned.

In [ ]:
import dciclient.v1.api.job as dci_job
def last_passing_puddle(ctx, topic_id):
    jobs = dci_job.iter(
        ctx,
        limit=100,
        embed='components',
        where='topic_id:' + topic_id)
    for job in jobs:
        if job['status'] == 'success':
            return(job['components'][0]['name'])

I've my data. I can prepare a data structure and generate a list.

In [ ]:
results = {}
for topic in topics:
    topic_id = topic_id_by_name(ctx, topic)
    results[topic] = [last_passing_puddle(ctx, topic_id)]

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(results, orient='index')
df.dtypes
df.columns = ['puddle']
df.head()

## Conclusion

It's pretty easy to use Python to quickly query the DCI server. If you have any question or comment you can contact use on #distributed-ci on FreeNode.

## References

* Our documentation: https://docs.distributed-ci.io/
* The OpenStack agent: http://github.com/redhat-cip/dci-ansible-agent
* The Control Server https://github.com/redhat-cip/dci-control-server